## 📐 Evaluation RQ3

Using **ARI (Adjusted Rand Index)** to measure the degree of overlap among the implemented approaches:

**1)** ARI Against the Ground-Truth.

**2)** Pairwise ARI Score.

#### Imports

In [ ]:
# Imports
from sklearn.metrics        import adjusted_rand_score
import matplotlib.pyplot    as plt
import seaborn              as sns
import pandas               as pd
import numpy                as np
import os

#### Parameters

In [ ]:
# Paths
INPUT_PATH = "../../0_Data/CSV/"

# Approaches
APPROACHES = ["3a_Drebin","3b_Dexray","3c_Rajasegaran","3d_Laficmil"]

In [ ]:
# Path 
PLOTS_PATH = "./Plots/"

# Create folder for paths
if not os.path.exists(PLOTS_PATH):
    os.makedirs(PLOTS_PATH)
    print("Folder created:", PLOTS_PATH)
else:
    print("Folder already exists:", PLOTS_PATH)

### Load Data

In [ ]:
appsDF = pd.read_csv(INPUT_PATH + "0_AndroCatSet.csv", index_col=False)
appsDF = appsDF.drop(['pkgName','googlePlayDescription'], axis=1)

# Add all the other approaches
for i in range(0,len(APPROACHES)):
    inputDF = pd.read_csv(INPUT_PATH + "RQ3/" + APPROACHES[i] + "ClusteringLabels.csv", index_col=False)
    appsDF = pd.concat([appsDF, inputDF['clusterID']], axis=1)
    appsDF = appsDF.rename(columns={'clusterID': APPROACHES[i][3:]})

### **1)**  ARI Against the Ground-Truth

Compute ARI Score.

In [ ]:
# To store ARI Score of each approach
ariScoreList = []

# Get approach List
approachesList = appsDF.columns[3:]

# For each approach compare against ground-truth
for approach in approachesList:
    #Compute the Adjusted Rand Index 
    ariScoreList.append(adjusted_rand_score(appsDF["classID"].values, appsDF[approach].values))

Plot ARI Score

In [ ]:
# Plot Parameters
TITLE_FONTSIZE = 18
TICKS_FONTSIZE = 14
AXIS_FONTSIZE  = 16

COLORS = ["#68B984"]

In [ ]:
fig = plt.figure(figsize=(7,4))

plt.bar(approachesList, ariScoreList, edgecolor='black',color=COLORS,width=0.4)

# Labels and Ticks
# Labels and Ticks
plt.xlabel('Approach' , fontsize=AXIS_FONTSIZE) 
plt.xticks(range(0,4), ['DREBIN','DexRay','Rajasegaran et al.','LaFiCMIL'],fontsize=TICKS_FONTSIZE)

plt.ylabel('ARI Score', fontsize=AXIS_FONTSIZE)  
plt.yticks(fontsize=TICKS_FONTSIZE)

plt.grid(axis='y', linestyle='--', alpha=0.7)

# add value labels to the top of the bars
for i in range(len(ariScoreList)):
    plt.text(i, ariScoreList[i]+0.01, "{:.2f}".format(ariScoreList[i]), ha='center', fontsize=TICKS_FONTSIZE)

plt.ylim(0,0.3)
plt.tight_layout() 

plt.savefig(PLOTS_PATH + 'overviewRQ3.png')
plt.savefig(PLOTS_PATH + 'overviewRQ3.pdf')

plt.show()

### **2)** Pairwise ARI Score.

Compute ARI score pairwise.

In [ ]:
# Get the approaches
approachesList = appsDF.columns[1:]

# Create a matrix for pairwise comparison
resultDF = pd.DataFrame(columns=approachesList,index=approachesList)

# Pairwise comparison
for i, (index, row) in enumerate(resultDF.iterrows()):
    for j in range(i,len(row)):
        #Compute the Adjusted Rand Index
        ariScore = adjusted_rand_score(appsDF[approachesList[i]].values, appsDF[approachesList[j]].values)
        
        # Store the results
        resultDF.loc[approachesList[i],approachesList[j]] = ariScore
        resultDF.loc[approachesList[j],approachesList[i]] = ariScore

resultDF = resultDF.astype(float)
resultDF